In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from scipy import stats
import matplotlib
import csv
import math
from matplotlib import pyplot as plt
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42

In [2]:
count=pd.read_excel('seahorse count.xlsx', index_col=0)
well=list(count['Fibroblast Count 1'].index)
welllist=[]
letterlist=['A','B','C','D','E','F','G','H']
for m in range(12):
    for k in range(8):
        if m<=8:
            welllist.append(letterlist[k]+'0'+str(m+1))
        if m>8:
            welllist.append(letterlist[k]+str(m+1))
Tcount=count.T
Tcount.columns=welllist
indexcount=Tcount.T

In [3]:
df=pd.read_excel('seahorse.xlsx', header=None)
replicates=['OCR1','ECAR1','OCR2','ECAR2','OCR3','ECAR3']
groups=['cd8','cd6','cd4','kd8','kd6','kd4','hek','hek-ih']
gcolumn=[[k for k in range(6)], [k+6 for k in range(6)], [k+12 for k in range(6)], [k+18 for k in range(6)], [k+24 for k in range(6)], [k+30 for k in range(6)], [k+36 for k in range(12)], [k+36 for k in range(24)]]

In [4]:
def dividematrix(replicate,group):
    plt.close('all')
    ndf=df.iloc[[k+(replicates.index(replicate)*13) for k in range(13)],:].dropna(axis=1).set_index(0).T.set_index(replicate)
    jdf=ndf.join(indexcount)
    dividelist=list(jdf['Fibroblast Count '+replicate[len(replicate)-1:]])
    matrixdivide=[]
    for k in range(len(dividelist)):
        matrixdivide.append([dividelist[k] for m in range(ndf.shape[1])])
    dmatrix=pd.DataFrame(ndf.to_numpy()/np.array(matrixdivide), columns=ndf.columns, index=ndf.index)
    fmatrix=dmatrix.iloc[gcolumn[groups.index(group)],:].T
    fnumpy=fmatrix.to_numpy()
    findex=fmatrix.index
    gmatrix=[]
    for k in range(len(fnumpy)):
        for m in range(len(fnumpy[k])):
            gmatrix.append([findex[k],fnumpy[k][m]])
    fdata=pd.DataFrame(gmatrix, columns=['Time',replicate[:len(replicate)-1]+'/Cells'])
    sns.lineplot(data=fdata, x='Time', y=replicate[:len(replicate)-1]+'/Cells')
    plt.savefig(replicate+' '+group+' seahorse.pdf')
    plt.close('all')

In [5]:
def combine(replicate,group):
    plt.close('all')
    ndf=df.iloc[[k+(replicates.index(replicate)*13) for k in range(13)],:].dropna(axis=1).set_index(0).T.set_index(replicate)
    jdf=ndf.join(indexcount)
    dividelist=list(jdf['Fibroblast Count '+replicate[len(replicate)-1:]])
    matrixdivide=[]
    for k in range(len(dividelist)):
        matrixdivide.append([dividelist[k] for m in range(ndf.shape[1])])
    dmatrix=pd.DataFrame(ndf.to_numpy()/np.array(matrixdivide), columns=ndf.columns, index=ndf.index)
    fmatrix=dmatrix.iloc[gcolumn[groups.index(group)],:].T
    fnumpy=np.mean(fmatrix.to_numpy(), axis=1)
    findex=fmatrix.index
    gmatrix=[]
    for k in range(len(fnumpy)):
        gmatrix.append([findex[k],fnumpy[k]])
    fdata=pd.DataFrame(gmatrix, columns=['Time',replicate[:len(replicate)-1]+'/Cells'])
    return(fdata)

In [6]:
ocrmatrix=[]
ecarmatrix=[]
timematrix=[]
for m in range(len(groups)-2):
    timeocr=np.zeros((1, 12))
    timeecar=np.zeros((1, 12))
    combineocr=[]
    combineecar=[]
    for k in range(3):
        timeocr+=combine(replicates[k*2],groups[m]).iloc[:,0].to_numpy()
        timeecar+=combine(replicates[(k*2)+1],groups[m]).iloc[:,0].to_numpy()
        combineocr.append(list(combine(replicates[k*2],groups[m]).iloc[:,1]))
        combineecar.append(list(combine(replicates[(k*2)+1],groups[m]).iloc[:,1]))
    timematrix.append((list((timeocr/3)[0])*3))
    ocrmatrix.append(sum(combineocr,[]))
    ecarmatrix.append(sum(combineecar,[]))
    #pdocr=pd.DataFrame([(list((zeroocr/3)[0])*3),sum(combineocr,[])]).T
    #pdecar=pd.DataFrame([(list((zeroocr/3)[0])*3),sum(combineecar,[])]).T

In [17]:
samplegroups=['Day 8', 'Day 6', 'Day 4']
plt.close('all')
plist=[['Group', 'Time','WT', 'KO', 'p-value', 'ko/wt FC']]
atpocr=[]
for m in range(3):
    gdatac=pd.DataFrame(zip(ocrmatrix[0+m], timematrix[0+m], ['WT' for k in range(len(timematrix[0+m]))])).rename(columns={1: "Time", 0: "OCR pmol/cell", 2: "Cell Type"})
    gdatak=pd.DataFrame(zip(ocrmatrix[3+m], timematrix[3+m], ['KO' for k in range(len(timematrix[3+m]))])).rename(columns={1: "Time", 0: "OCR pmol/cell", 2: "Cell Type"})
    sns.lineplot(data=gdatac.append(gdatak, ignore_index=True), x="Time", y="OCR pmol/cell", hue='Cell Type', markers=True, style='Cell Type')
    plt.savefig(samplegroups[m]+' OCR combined seahorse.pdf')
    plt.close('all')
    for k in range(12):
        ocrwt=np.transpose(np.array(ocrmatrix[0+m]).reshape(3,12))[k]
        ocrko=np.transpose(np.array(ocrmatrix[3+m]).reshape(3,12))[k]
        atpocr.append(list(ocrwt)+list(ocrko))
        plist.append([samplegroups[m]+' OCR', timematrix[0][k], ocrwt, ocrko, stats.ttest_rel(ocrwt, ocrko)[1], (np.mean(ocrko)/np.mean(ocrwt))])
atpecar=[]
for m in range(3):
    gdatac=pd.DataFrame(zip(ecarmatrix[0+m], timematrix[0+m], ['WT' for k in range(len(timematrix[0+m]))])).rename(columns={1: "Time", 0: "ECAR mpH/cell", 2: "Cell Type"})
    gdatak=pd.DataFrame(zip(ecarmatrix[3+m], timematrix[3+m], ['KO' for k in range(len(timematrix[3+m]))])).rename(columns={1: "Time", 0: "ECAR mpH/cell", 2: "Cell Type"})
    sns.lineplot(data=gdatac.append(gdatak, ignore_index=True), x="Time", y="ECAR mpH/cell", hue='Cell Type', markers=True, style='Cell Type')
    plt.savefig(samplegroups[m]+' ECAR combined seahorse.pdf')
    plt.close('all')
    for k in range(12):
        ecarwt=np.transpose(np.array(ecarmatrix[0+m]).reshape(3,12))[k]
        ecarko=np.transpose(np.array(ecarmatrix[3+m]).reshape(3,12))[k]
        atpecar.append(list(ecarwt)+list(ecarko))
        plist.append([samplegroups[m]+' ECAR', timematrix[0][k], ecarwt, ecarko, stats.ttest_rel(ecarwt, ecarko)[1], (np.mean(ecarko)/np.mean(ecarwt))])
atpmatrix=(np.array(atpocr)*2.8)+(np.array(atpecar)*1.4)
atpfinal=[]
for m in range(len(atpmatrix)):
    for n in range(3):
        atpfinal.append([atpmatrix[m][n], timematrix[0][m], 'WT'])
        atpfinal.append([atpmatrix[m][n+3], timematrix[0][m], 'KO'])
for p in range(3):
    atpg=pd.DataFrame(atpfinal[(0+(72*p)):(72+(72*p))]).rename(columns={1: "Time", 0: "ATP (pmol+mpH)/cell", 2: "Cell Type"})
    sns.lineplot(data=atpg, x="Time", y="ATP (pmol+mpH)/cell", hue='Cell Type', markers=True, style='Cell Type')
    plt.savefig(samplegroups[p]+' ATP combined seahorse.pdf')
    plt.close('all')
    sns.barplot(data=atpg[(6*6):len(atpg)-(6*3)], x="Time", y="ATP (pmol+mpH)/cell", hue='Cell Type')
    plt.savefig(samplegroups[p]+' ATP combined barplot.pdf')
    plt.close('all')
for q in range(len(atpmatrix)):
    atpwt=atpmatrix[q][:3]
    atpko=atpmatrix[q][3:]
    plist.append([samplegroups[math.floor(q/12)]+' ATP', timematrix[0][q], atpwt, atpko, stats.ttest_rel(atpwt, atpko)[1], (np.mean(atpko)/np.mean(atpwt))])
with open('seahorse pvalue.csv', 'w', newline='') as file:
    for k in range(len(plist)):
        csv.writer(file).writerow(plist[k])

In [80]:
gdatac.append(gdatak, ignore_index=True)

,ECAR/Cells,Time,Cell Type
0,0.003443,1.310000,WT
1,0.002985,7.786667,WT
2,0.002784,14.256667,WT
3,0.003333,20.823333,WT
4,0.001544,27.300000,WT
...,...,...,...
67,0.003834,46.820000,KO
68,0.003601,53.300000,KO
69,0.002071,59.866667,KO
70,0.001568,66.343333,KO


In [68]:
gdatak

,ECAR/Cells,Time,Cell Type
0,0.003227,1.310000,KO
1,0.002703,7.786667,KO
2,0.002431,14.256667,KO
3,0.003198,20.823333,KO
4,0.001571,27.300000,KO
5,0.001349,33.776667,KO
6,0.004543,40.343333,KO
7,0.004065,46.820000,KO
8,0.003985,53.300000,KO
9,0.003000,59.866667,KO
